# RPA -  Python

A demo of a simple notebook that fires automatic e-mails after downloading and processing an Excel file from google drive.
It uses pyautogui (https://pyautogui.readthedocs.io/en/latest/) and pandas (https://pandas.pydata.org/)
for command inputs and data handling, respectively. 

For the demo you can set the parameters interactively:
  
    - google drive directory link
    - receiver of the e-mail
    - e-mail text template (if not specified uses the default from the repo)
    - columns of the excel file for processing 
    
    
Also, it is important to note that, the tool assumes you have your browser open, and you have access to the specified folder.


# Config 



In [44]:
import os.path
import time

template_email_path = './resource/template.txt'


# validation
if(not(template_email_path)):
    raise Exception('Template path invalid, please verify')

# app-info
your_name = 'Mateus'
directory_link_default = 'https://drive.google.com/drive/folders/1yShrodfbIuSqfb4A60yoJj9dumeI1nZt?usp=sharing'
receivers = ['email1@email.com', 'email2@email.com', 'email3@email.com'] # set-up more e-mails puting and overwrite those for the ones you want
columns = ['Valor Unitário', 'Valor Final'] # you can also set-up more columns or overwrite those

# location of the image of buttons that program should press
folder_image_path = './resource/folder-name.png' 
download_image_path1 = './resource/download1.png' 
download_image_path2 = './resource/download2.png'
download_image_path3 = './resource/download3.png'
compose_image_path = './resource/compose.png' 

#dict to cache the locations
locations ={ 
    "folder": { 
        "pos" : None
    }, 
    "download1" : {
        "pos" : None
    }, 
    "download2" : {
        "pos" : None
    }, 
    "compose" : {
        "pos" : None
    } 
} 

# cache in a local file the button locations if those are not found
def cacheLocations(location, BUTTON_ID):
    f = None
    if(os.path.isfile('./resource/locations')):
        f = open('./resource/locations', 'w')     
    else:
        f = open('./resource/locations', 'a')
    f.write(BUTTON_ID)
    #f.write(location)
    f.close()
        
    
# locate button handler
def locateButton(imagePath, BUTTON_ID):
    location = None
    try:
        if(os.path.isfile(imagePath)):
            location = pyautogui.locateOnScreen(imagePath)
            if(location == None):
                raise Exception()
            else:
                return location
        else:
            raise Exception()
    except:
        pyautogui.alert('Unable to auto-locate the button, please click on the ok button and put the mouse over the the correct spot for 10 seconds the screen.\nThe location will be saved for the next interactions')
        time.sleep(10)
        location = pyautogui.position()
        #cacheLocations(location, BUTTON_ID)
        return location

# 1 :  Detecting buttons and opening drive directory 

In [38]:
import pyautogui
import pyperclip

# delay config
pyautogui.PAUSE = 1

pyautogui.hotkey('ctrl', 't')
pyperclip.copy(directory_link_default)
pyautogui.hotkey('ctrl', 'v')
pyautogui.press('enter')

time.sleep(5)
button_positionX, button_positionY  = locateButton(folder_image_path, 'FOLDER_BUTTON')
pyautogui.click(x=button_positionX, y=button_positionY, clicks=2)


# 2 :  Detecting button and downloading file

In [41]:
time.sleep(2)
#d1_positionX, d1_positionY  = locateButton(folder_image_path, 'DOWNLOAD1')
d1_positionX, d1_positionY  = 452, 355
pyautogui.click(x=d1_positionX, y=d1_positionY)

#d2_positionX, d2_positionY  = locateButton(folder_image_path, 'DOWNLOAD2')
d2_positionX, d2_positionY  = 1161, 185
pyautogui.click(x=d2_positionX, y=d2_positionY)

#d3_positionX, d3_positionY  = locateButton(folder_image_path, 'DOWNLOAD3')
d3_positionX, d3_positionY  = 1057, 583
pyautogui.click(x=d3_positionX, y=d3_positionY)

time.sleep(10)

pyautogui.press('enter')
pyautogui.press('enter')

# 3 :  Data Manipulation 

In [42]:
import pandas

# TODO: prev step should choose directory
data_frame = pandas.read_excel(r"C:\Users\mateu\Downloads\Vendas - Dez.xlsx")
results = []

results = [data_frame[column].sum() for column in columns ]    


# 4 :  Formating the E-mail Template

In [45]:
template_msg_file = open(template_email_path, 'r')
subject = ''
msg = ''

counter = 0
for line in template_msg_file:
    if(counter > 1):
        msg += line
    elif(counter == 0):
        subject = line[:-1]
    counter+= 1

# 5 :  Sending the E-mail

In [ ]:
pyautogui.hotkey('ctrl', 't')
pyperclip.copy('https://mail.google.com/mail/u/0/')
pyautogui.hotkey('ctrl', 'v')
pyautogui.press('enter')
time.sleep(5)    
    
final_msg = msg.format(faturamento=results[1], qtdProd=results[0], yourName=your_name)

compose_positionX, compose_positionY  = 144, 188
pyautogui.click(x=compose_positionX, y=compose_positionY)

for receiver in receivers:
    pyautogui.write(receiver)
    pyautogui.write(' ')
    if(len(receivers) == 1):
        pyautogui.press('tab')
        

pyautogui.press('tab')
pyperclip.copy(subject)
pyautogui.hotkey('ctrl', 'v')

pyautogui.press('tab')
pyperclip.copy(final_msg)
pyautogui.hotkey('ctrl', 'v')
pyautogui.hotkey('ctrl', 'enter')

In [31]:
# while the handler is not complete 
time.sleep(10)
pyautogui.position()

Point(x=336, y=429)